In [152]:
import pandas as pd
import string
import openpyxl
import datetime
import random
import os

In [153]:
# Assuming 'User_database.xlsx' is in the same directory as the notebook/script
excel_file1 = '/kaggle/input/d/mugdhazadkar/user-database/User_database.xlsx'
df_user = pd.read_excel(excel_file1)

# Create key database
headers_1 = ['key_label', 'key_value', 'user_id', 'session_id', 'key_length', 'key_domain', 'generation_datetime']
df_keys = pd.DataFrame(columns=headers_1)

#Create key usage database
headers_2 = ['key_label', 'user_id', 'session_id', 'datetime']
df_key_usage = pd.DataFrame(columns=headers_2)

#Create KMS logs
headers_3 = ['key_label', 'user_id', 'session_id', 'datetime', 'activity', 'result']
df_key_logs = pd.DataFrame(columns=headers_3)

In [154]:
df_key_logs

,key_label,user_id,session_id,datetime,activity,result


In [155]:
#user_authentication function

def authenticate_user(username, password):
    user_match = df_user[(df_user['username'] == username) & (df_user['password'] == password)]
    
    if not user_match.empty:
        return 'User authenticated' #It should return the user object called from user management
    else:
        return 'User not recognized'

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

# Authenticate the user
result = authenticate_user(username, password)
print(result)

Enter your username:  
Enter your password:  


User not recognized


In [156]:
# replacement for Gokul's function
def random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

# Function to convert bytes to hexadecimal
def bytes_to_hex(data):
    return ''.join(['{:02x}'.format(byte) for byte in data])

#key generation function
def key_generation(user_id, session_id, key_length, key_domain):
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key_value = random_string(key_length)
    key_label = random.randint(10000, 99999)  # Generate a key label
    key_value_hex = bytes_to_hex(key_value.encode('utf-8'))
    
    #create a dictionary with the required data
    new_key =  pd.DataFrame({'key_label':[key_label], 'key_value':[key_value_hex], 'user_id':[user_id], 'session_id':[session_id], 'key_length':[key_length], 'key_domain':[key_domain], 'generation_datetime':[generation_datetime]})
  
    # Use df.append to add the new data entry to the DataFrame
    global df_keys
    df_keys = pd.concat([df_keys, new_key], ignore_index=True)
    print(f"Key generated with key label {key_label}")
    
    #update logs
    global df_key_logs
    activity = 'key generation'
    result = 'success'
    new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
    df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
    return df_keys


In [157]:
def key_sharing(key_label, user_id, session_id):
    #the session_ID here is to record the session during which the key was shared. 
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    global df_keys
    global df_key_logs
    user_key_exists = df_keys[(df_keys['user_id'] == user_id) & (df_keys['key_label'] == key_label)]
    if not user_key_exists.empty:
        # Finding the row and column number
        row_number = user_key_exists.index[0]
        #row_number = df_keys.index[df_keys['key_label'] == key_label].tolist()
        column_name = "key_value"
        column_number = df_keys.columns.get_loc(column_name)
        
        # finding a particular key value based on row number and column number
        key_value = df_keys.iloc[row_number, column_number]
        global df_key_usage
        new_usage = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime]})
        df_key_usage = pd.concat([df_key_usage, new_usage], ignore_index=True)
        print(f"The requested key is {key_value}")
        
         #update logs
        activity = 'key sharing'
        result = 'success'
        new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
        df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
        
        return df_key_usage
    else:
        #update logs
        activity = 'key sharing'
        result = 'failure'
        new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
        df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
        print("Please check the user ID and key label.")

In [158]:
# Function to delete a key from the key database
def key_termination(key_label):
    global df_key_logs
    global df_keys
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        key_label_exists = key_label in df_keys['key_label'].values
        if key_label_exists:
        # Find the row with the specified key_label
            df_keys = df_keys.drop(df_keys[df_keys['key_label'] == key_label].index)
            
             #update logs
            activity = 'key termination'
            result = 'success'
            new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
            df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
            
            return df_keys
        else: 
            print(f"Key label '{key_label}' not found in the database.")
            #update logs      
            activity = 'key termination'
            result = 'failure'
            new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
            df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
            
            return df_keys
    except Exception as e:
        #update logs      
        activity = 'key termination'
        result = 'failure'
        new_activity = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'datetime':[generation_datetime], 'activity':[activity], 'result':[result]})
        df_key_logs = pd.concat([df_key_logs, new_activity], ignore_index=True)
        print(f"An error occurred: {str(e)}")

In [159]:
# Example usage of the key_generation function
user_id = "mugdha"
session_id = 101
key_length = 16
key_domain = "QURE-derived"

key_generation(user_id, session_id, key_length, key_domain)


Key generated with key label 44731


,key_label,key_value,user_id,session_id,key_length,key_domain,generation_datetime
0,44731,4b4a545750477a6c5830594b79426544,mugdha,101,16,QURE-derived,2023-10-11 12:13:20


In [160]:
delete_key = 15881
key_termination(delete_key)

Key label '15881' not found in the database.


,key_label,key_value,user_id,session_id,key_length,key_domain,generation_datetime
0,44731,4b4a545750477a6c5830594b79426544,mugdha,101,16,QURE-derived,2023-10-11 12:13:20


In [161]:
key_label = 20585
user_id = "mugdha"
session_id = 101
key_sharing(key_label, user_id, session_id)
df_key_usage

Please check the user ID and key label.


,key_label,user_id,session_id,datetime


In [162]:
df_key_logs

,key_label,user_id,session_id,datetime,activity,result
0,44731,mugdha,101,2023-10-11 12:13:20,key generation,success
1,15881,mugdha,101,2023-10-11 12:13:20,key termination,failure
2,20585,mugdha,101,2023-10-11 12:13:20,key sharing,failure
